In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
# pd.set_option('display.min_rows',0) # less likely helpful except specific circumstance
# pd.set_option('display.max_rows',None) 

##### Check scraped 700 pages

In [ ]:
df = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq.csv')

In [ ]:
df.shape[0]/15 #15 projects per page


In [ ]:
df.shape[0]

###### Rescraped to include try except=None on backers

In [ ]:
df2 = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq2.csv')

In [ ]:
df2.shape[0]/15 # check if 700 pages

##### Check to see if any backers null

In [ ]:
df2.loc[df2['backers'].isnull()] #no nulls

##### Check to see if two scrapes returned same number rows

In [ ]:
df2.shape[0] == df.shape[0]

# Transformation

In [106]:
# Using df2 reassign as dfn for newly scraped kicktraq - to distinguish between older kaggle dataset (dfo)
dfn = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq2.csv')

In [107]:
dfn.sample(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status
3741,58,photography|fine art,$,"Exotica, a beautiful photobook featuring the f...",2020.0,photography,Exotica: Devi Art Nudes Photo Book,3005.0,"\n\t\t\t\t\tBackers: 58,\n\t\t\t\t\tFunding: $...",Closed
9547,88,art|illustration,$,From the creator of Oddity Tarot comes a Noteb...,1050.0,art,Lucky Potion Oracle Companion,5577.0,"\n\t\t\t\t\tBackers: 88,\n\t\t\t\t\tFunding: $...",Closed
8486,40,publishing|children's books,$,A children’s book about a dragon’s quest to ov...,10000.0,publishing,Drake’s Quest for Life-A book of hope for depr...,1220.0,"\n\t\t\t\t\tBackers: 40,\n\t\t\t\t\tFunding: $...",Closed
1514,1,publishing|nonfiction,$,I have compiled two books which I am planning ...,75000.0,publishing,The Naked 'I' - Behind The Cloak Of The Visibl...,1.0,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: $1...",Cancelled
2135,4,music|kids,$,Children's songs,100.0,music,The Godino Show,131.0,"\n\t\t\t\t\tBackers: 4,\n\t\t\t\t\tFunding: $1...",Closed


In [108]:
dfn.rename({'names':'name'}, axis = 1, inplace=True)

In [109]:
# dropping some columns, will get from proj_details with more accuracy
dfn = dfn.drop(['currency', 'goal', 'pledged'], axis=1)

### Splitting category

In [110]:
# split on '|' make new dfn_cat
dfn_cat = dfn['category'].str.split('|',expand=True) # merge to data frame

In [111]:
# concat dfn and dfn_cat, drop 0 (name of the first component of split), rename second new column 'sub_category'
dfn = pd.concat([dfn, dfn_cat], axis=1).drop(0,1).rename(columns = {1:'sub_category'})

In [112]:
dfn.sample(5)

,backers,category,description,main_category,name,proj_details,status,sub_category
4260,111,games|tabletop games,The Fighting Board Game that feels like an Arc...,games,Dragon Fighters: Advanced Musha Shugyo RPG,"\n\t\t\t\t\tBackers: 111,\n\t\t\t\t\tFunding: ...",Closed,tabletop games
6564,12,technology|space exploration,De San Luis para el Mundo,technology,Observatorio Astronómico con Acceso remoto,"\n\t\t\t\t\tBackers: 12,\n\t\t\t\t\tFunding: M...",Closed,space exploration
2838,555,games|video games,Major Expansion and PC Steam Release for Pixel...,games,Pixel Starships: Galaxy,"\n\t\t\t\t\tBackers: 555,\n\t\t\t\t\tFunding: ...",Closed,video games
1553,176,games|playing cards,Impress cardistry with visual effects. \n 100%...,games,Mizutama Cards SPECTRUM Ed.,"\n\t\t\t\t\tBackers: 176,\n\t\t\t\t\tFunding: ...",Closed,playing cards
7984,42,music|kids,I will be starting a music class for babies &...,music,Baby/Toddler Music Class,"\n\t\t\t\t\tBackers: 42,\n\t\t\t\t\tFunding: $...",Closed,kids


##### Extracting date portion of proj_details into 'campaign_dates' col - there are no missing values 

In [113]:
### KEEP THIS! THIS GETS campaign dates as a chunck will play with to get datetimes
dfn['campaign_dates'] = [re.findall('Campaign (Dates: .+)', el)[0] for el in dfn['proj_details']]    

##### 'launched' col from campaign_dates

In [114]:
# creates list of tuples with Month, day , year
start_date = [re.findall('Dates: (\D*) (\d*).+\((\d+)\)', el)[0] for el in dfn['campaign_dates']]

In [115]:
# join tuple elements into string, now list of strings
start_date = [' '.join(date) for date in start_date]

In [116]:
# create column 'launched' with datetime object from list
dfn['launched']= [pd.datetime.strptime(s, '%B %d %Y') for s in start_date]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [117]:
dfn.head(5)

,backers,category,description,main_category,name,proj_details,status,sub_category,campaign_dates,launched
0,10,food|drinks,A do it yourself boba tea kit that allows you ...,food,Boba Era: Boba From Home,"\n\t\t\t\t\tBackers: 10,\n\t\t\t\t\tFunding: $...",Cancelled,drinks,"Dates: June 3rd -> August 2nd (2020),",2020-06-03
1,1,fashion|apparel,Sophisticated Mexican old school style apparel.,fashion,Mr Cochino Apparel,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: $1...",Cancelled,apparel,"Dates: June 26th -> August 2nd (2020),",2020-06-26
2,1,fashion|apparel,RotenStein Clothing Co. Una nueva marca de rop...,fashion,RotenStein Clothing Co. (Arte en tu ropa),"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: MX...",Cancelled,apparel,"Dates: June 3rd -> August 2nd (2020),",2020-06-03
3,1,games|video games,"Z-City, A Zombie based Survival-Battle Royale ...",games,Z-City Survival-Battle Royale,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: $1...",Cancelled,video games,"Dates: July 3rd -> August 2nd (2020),",2020-07-03
4,0,comics|graphic novels,A Samurai Horror Graphic Novel,comics,Death Kanji,"\n\t\t\t\t\tBackers: 0,\n\t\t\t\t\tFunding: $0...",Cancelled,graphic novels,"Dates: July 3rd -> August 2nd (2020),",2020-07-03


##### 'deadline' same as 'launched' above

In [118]:
end_date = [re.findall('-> (\D+) (\d+).+\((\d+)\)', el)[0] for el in dfn['campaign_dates']]

In [119]:
end_date = [' '.join(date) for date in end_date]

In [120]:
# drop campaign dates
dfn['deadline']= [pd.datetime.strptime(s, '%B %d %Y') for s in end_date]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [121]:
dfn.sample(5)

,backers,category,description,main_category,name,proj_details,status,sub_category,campaign_dates,launched,deadline
1449,2,design|product design,Eco-Friendly | Traditionally Sourced | Turkish...,design,COTON | Ethically Hand Made 100% Turkish Cotto...,"\n\t\t\t\t\tBackers: 2,\n\t\t\t\t\tFunding: $1...",Cancelled,product design,"Dates: March 17th -> April 16th (2020),",2020-03-17,2020-04-16
8031,3,games|mobile games,Challenge your math skills!,games,Do Plus!,"\n\t\t\t\t\tBackers: 3,\n\t\t\t\t\tFunding: €2...",Closed,mobile games,"Dates: May 27th -> June 30th (2020),",2020-05-27,2020-06-30
6454,26,publishing|young adult,"An anxious werewolf king, with a town to prote...",publishing,Blackwood Volume 1 Printing,"\n\t\t\t\t\tBackers: 26,\n\t\t\t\t\tFunding: $...",Closed,young adult,"Dates: May 12th -> June 13th (2020),",2020-05-12,2020-06-13
5411,2,technology|fabrication tools,My wish is to make a roller cleaning machine a...,technology,Coloring is fun again The cleaning and wetting...,"\n\t\t\t\t\tBackers: 2,\n\t\t\t\t\tFunding: €6...",Closed,fabrication tools,"Dates: May 3rd -> June 2nd (2020),",2020-05-03,2020-06-02
10467,5,publishing|children's books,"A fully illustrated children’s book, written i...",publishing,Jack’s Secret Garden,"\n\t\t\t\t\tBackers: 5,\n\t\t\t\t\tFunding: £6...",Cancelled,children's books,"Dates: June 2nd -> July 31st (2020),",2020-06-02,2020-07-31


##### Extracting funding portion of proj_details into 'funding' col also deals with missing values

In [122]:
##### KEEP THIS HOW WE GOT THE FUNDING STRING EVEN IF MISSING
dfn['funding'] = [None if re.findall('Funding: (.*)', el)==[] else re.findall('Funding: (.*)', el)[0] for el in dfn['proj_details']] 

##### Spliting funding into three columns: currency, pledged, goal

In [123]:
dfn['funding'] = dfn['funding'].str.replace(',','')

In [124]:
# turn funding into tuple (currency, pledged, goal) take into account None types in list comprehension
dfn['funding'] = [None if el==None else re.findall('(\D*)(\d*) of \D*(\d*) \(', el)[0] for el in dfn['funding']] 

In [125]:
# splitting tuple into columns in new dfn_fund
dfn_fund = dfn['funding'].apply(pd.Series)

In [126]:
# renaming columns
dfn_fund.columns=['currency_sym', 'pledged', 'goal']

In [127]:
dfn_fund.head(5)

,currency_sym,pledged,goal
0,$,401,3000
1,$,1,16578
2,MX$,19,73000
3,$,1,5000
4,$,0,7500


In [128]:
# concat dfn and dfn_fund and drop unnecessary columns
dfn = pd.concat([dfn, dfn_fund], axis = 1).drop(['proj_details', 'category', 'campaign_dates', 'funding'], axis = 1)

In [129]:
dfn.head(5)

,backers,description,main_category,name,status,sub_category,launched,deadline,currency_sym,pledged,goal
0,10,A do it yourself boba tea kit that allows you ...,food,Boba Era: Boba From Home,Cancelled,drinks,2020-06-03,2020-08-02,$,401,3000
1,1,Sophisticated Mexican old school style apparel.,fashion,Mr Cochino Apparel,Cancelled,apparel,2020-06-26,2020-08-02,$,1,16578
2,1,RotenStein Clothing Co. Una nueva marca de rop...,fashion,RotenStein Clothing Co. (Arte en tu ropa),Cancelled,apparel,2020-06-03,2020-08-02,MX$,19,73000
3,1,"Z-City, A Zombie based Survival-Battle Royale ...",games,Z-City Survival-Battle Royale,Cancelled,video games,2020-07-03,2020-08-02,$,1,5000
4,0,A Samurai Horror Graphic Novel,comics,Death Kanji,Cancelled,graphic novels,2020-07-03,2020-08-02,$,0,7500


### Currency - adding currency code and USD exchange rate

In [130]:
dfn.loc[dfn['currency_sym']=='kr']

,backers,description,main_category,name,status,sub_category,launched,deadline,currency_sym,pledged,goal
49,3,A book about dimension mapping and meetings wi...,publishing,Clockwork Elves and DMT mapping,Cancelled,NaN,2020-07-08,2020-08-07,kr,289,16700
209,14,A small series of elegant holders in a Scandin...,design,Edge Holders,Closed,product design,2020-06-18,2020-07-18,kr,4862,60000
266,13,We are recording our debut album.,music,Communist Sex: Lada Riva,Closed,indie rock,2020-02-05,2020-04-05,kr,3089,3000
327,10,Reborn project of Swedish 80's metal band Ivory.,music,Ivory Reborn - Swedish 80's metal. Part 1 – di...,Closed,metal,2020-02-25,2020-04-05,kr,3879,75000
338,1,The world's first Social Media that allows you...,technology,THE DABO APP: Challenge the world,Cancelled,apps,2020-02-26,2020-04-06,kr,8,800000
...,...,...,...,...,...,...,...,...,...,...,...
10297,841,Epic narrative adventure board game for 1-4 pl...,games,"Unlikely Heroes: Vikings, Cowboys & Pharaohs",Closed,tabletop games,2020-06-23,2020-07-23,kr,367794,10000
10398,4,I have created a luxury inspired watch design ...,fashion,"Marshall's Ace - ""Eye Ticking"" Watch Brand Fro...",Cancelled,accessories,2020-06-22,2020-07-24,kr,5105,170000
10402,2,A Visual Music Album within the Pop Genre to e...,music,'The first breath of a Rainbow (A Visual Music...,Cancelled,pop,2020-05-25,2020-07-24,kr,259,170000
10415,82,The 2 in 1 vase that brings security and safet...,design,The FireVase,Cancelled,NaN,2020-05-27,2020-07-26,kr,45042,249355


##### Changing currency

In [131]:
dfn['currency_sym'].value_counts() # see what currency we have

$      6661
€      1560
£      1446
MX$     292
kr      232
HK$     220
¥        71
Name: currency_sym, dtype: int64

##### creating currency exchange data frame

In [132]:
symb_dict = {'currency_sym':['$','€','£','¥','MX$','kr','HK$'], 'currency':['USD', 'EUR', 'GPB', 'JPY', 'MXN', 'DKK', 'HKD'], 'exchange_rt_to_USD': [1, 1.16, 1.28, 0.0094, 0.045, 0.16, 0.13]}
df_cur_sym = pd.DataFrame(symb_dict)
df_cur_sym


,currency_sym,currency,exchange_rt_to_USD
0,$,USD,1.0000
1,€,EUR,1.1600
2,£,GPB,1.2800
3,¥,JPY,0.0094
4,MX$,MXN,0.0450
5,kr,DKK,0.1600
6,HK$,HKD,0.1300


##### merge dfn with currency exchange

In [133]:
dfn = pd.merge(dfn, df_cur_sym, how='left', on='currency_sym')

In [134]:
dfn.sample(5)

,backers,description,main_category,name,status,sub_category,launched,deadline,currency_sym,pledged,goal,currency,exchange_rt_to_USD
7211,275,Alchemy of time!,games,Tarot of Unknown Shadows,Closed,playing cards,2020-06-05,2020-06-21,€,9538,3000,EUR,1.160
8062,220,Massachusetts calls 4 Massive Witches + Mutant...,comics,Faro's American Safari -- MASSACHUSETTS EDITIO...,Closed,anthologies,2020-06-03,2020-06-30,$,13797,5000,USD,1.000
6733,751,Super tasty. Nutritionally Complete. Plant bas...,food,Future Noodles,Closed,vegan,2020-05-18,2020-06-17,£,55050,50000,GPB,1.280
7991,2,The tool you did not know you needed,technology,MULTITOOL ARM EXTENTION,Closed,fabrication tools,2020-05-11,2020-06-30,MX$,3019,250000,MXN,0.045
9893,32,A multimedia remote control built to fit on th...,design,ergRemote,Closed,product design,2020-06-18,2020-07-18,$,4002,47000,USD,1.000


#### creating columns usd_pledged and usd_goal

In [135]:
dfn['pledged']= round(pd.to_numeric(dfn["pledged"], downcast="float"),2)

In [136]:
dfn['usd_pledged'] = dfn['pledged']*dfn['exchange_rt_to_USD']

In [137]:
dfn['goal'] = round(pd.to_numeric(dfn['goal'], downcast="float"),2)

In [138]:
dfn['usd_goal'] = dfn['goal']*dfn['exchange_rt_to_USD']

In [139]:
dfn['backers'] = pd.to_numeric(dfn['backers'])

In [140]:
dfn.sample(2)

,backers,description,main_category,name,status,sub_category,launched,deadline,currency_sym,pledged,goal,currency,exchange_rt_to_USD,usd_pledged,usd_goal
4850,23,A set of detailed printable terrain and our 5e...,games,Into The Witchelm: Module and Terrain,Closed,tabletop games,2020-04-26,2020-05-27,$,640.0,250.0,USD,1.00,640.00,250.0
6768,16,"This is a film about the 5-year-old Caroline, ...",film & video,"""Mommy, Why Is Santa Crying? "" | Short film",Closed,shorts,2020-05-18,2020-06-17,kr,15456.0,10000.0,DKK,0.16,2472.96,1600.0


In [74]:
# dfn['pledged'], dfn['goal'], dfn['backers'] = pd.to_numeric(dfn["pledged"], dfn['goal'], dfn['backers'], downcast="float")

In [141]:
# create column p_funded percent funded for each project
dfn['p_funded'] = round((dfn['pledged']/dfn['goal'])*100, 2)

In [147]:
dfn['p_funded']

0        13.37
1         0.01
2         0.03
3         0.02
4         0.00
         ...  
10490     0.01
10491     0.00
10492     0.00
10493     0.34
10494    62.25
Name: p_funded, Length: 10495, dtype: float32

##### subseting data and creating boolean column successful 

In [157]:
dfn['successful'] = dfn['p_funded']>=100

In [159]:
dfn_closed = dfn.loc[dfn['status']=='Closed']

In [173]:
dfn_closed.head(3)

,backers,description,main_category,name,status,sub_category,launched,deadline,currency_sym,pledged,goal,currency,exchange_rt_to_USD,usd_pledged,usd_goal,p_funded,successful
195,52,Everything You Need to Know About My Cat’s Car...,publishing,What About The Cat? (The book!),Closed,NaN,2020-06-18,2020-07-18,$,1678.0,700.0,USD,1.00,1678.00,700.0,239.710007,True
196,8,A Virtual Reality Horror Theme Park Video Game,games,Terror Fest,Closed,video games,2020-05-20,2020-07-18,$,182.0,40000.0,USD,1.00,182.00,40000.0,0.460000,False
197,2,"An anthology series of 6 episodes, each with a...",film & video,Our Town: An anthology web series,Closed,webseries,2020-06-18,2020-07-18,£,6.0,5000.0,GPB,1.28,7.68,6400.0,0.120000,False


### Kaggel dataframe loaded as dfo

In [211]:
dfo = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/data/Web_Scraping_Proj/ks-projects-201801.csv')
dfo.sample(3)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
99693,1506196682,SimpL Wallet - Just the Essentials - CNC machi...,Product Design,Design,USD,2016-07-20,5000.0,2016-06-20 17:00:08,2541.0,failed,55,US,1660.0,2541.0,5000.00
329516,74889044,Killer Words - Independent Short Film,Thrillers,Film & Video,USD,2015-01-18,5000.0,2014-12-19 20:24:00,0.0,failed,0,US,0.0,0.0,5000.00
23643,1120095924,Around Da World In 3D Waves,Shorts,Film & Video,CAD,2014-03-08,850.0,2014-02-06 09:18:17,0.0,failed,0,CA,0.0,0.0,766.66


In [42]:
dfo[['category', 'main_category']]

,category,main_category
0,Poetry,Publishing
1,Narrative Film,Film & Video
2,Narrative Film,Film & Video
3,Music,Music
4,Film & Video,Film & Video
...,...,...
378656,Documentary,Film & Video
378657,Narrative Film,Film & Video
378658,Narrative Film,Film & Video
378659,Technology,Technology


In [246]:
# renameing to match dfn. category here is sub_category above
# note that instead of NaN like dfn for subcat that do not exist, here they repeated info in main_cat column
dfo.rename({'usd_pledged_real':'usd_pledged', 'usd_goal_real':'usd_goal', 'category':'sub_category'}, axis=1, inplace=True)

In [214]:
# again to match dfn 
dfo['sub_category'] = dfo['sub_category'].str.lower()
dfo['main_category'] = dfo['main_category'].str.lower()

In [215]:
#
dfo.drop('usd pledged', axis=1, inplace=True)

In [216]:
# making p_funded
dfo['p_funded'] = round((dfo['pledged']/dfo['goal'])*100, 2)

In [217]:
# making successful boolean column
dfo['successful'] = dfo['p_funded']>=100

In [218]:
dfo['state'].value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [219]:
# limiting data to failed and successful like dfn
dfo_closed = dfo.loc[(dfo['state']=='failed') | (dfo['state']=='successful')]

In [220]:
dfo_closed['state'].value_counts()

failed        197719
successful    133956
Name: state, dtype: int64

### Concat the dfs

In [227]:
df = pd.concat([dfn_closed, dfo_closed], join='inner', ignore_index=True)

In [242]:
df.tail(5)

,backers,main_category,name,sub_category,launched,deadline,pledged,goal,currency,usd_pledged,usd_goal,p_funded,successful
340348,4,food,"Homemade fresh dog food, Cleveland OH",small batch,2017-03-20 22:08:22,2017-04-19,154.0,6500.0,USD,154.0,6500.0,2.37,False
340349,5,film & video,The Tribe,narrative film,2011-06-22 03:35:14,2011-07-19,155.0,1500.0,USD,155.0,1500.0,10.33,False
340350,1,film & video,Walls of Remedy- New lesbian Romantic Comedy f...,narrative film,2010-07-01 19:40:30,2010-08-16,20.0,15000.0,USD,20.0,15000.0,0.13,False
340351,6,technology,BioDefense Education Kit,technology,2016-01-13 18:13:53,2016-02-13,200.0,15000.0,USD,200.0,15000.0,1.33,False
340352,17,art,Nou Renmen Ayiti! We Love Haiti!,performance art,2011-07-19 09:07:47,2011-08-16,524.0,2000.0,USD,524.0,2000.0,26.20,False


In [245]:
dfn_closed.shape[0]+dfo_closed.shape[0] == df.shape[0]

True